In [1]:
%reload_ext Cython
%reload_ext line_profiler

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c, e, m_p
from scipy.signal import fftconvolve

%matplotlib tk
import seaborn as sns
sns.set_context('notebook', font_scale=1.5,
                rc={'lines.markeredgewidth': 1})
sns.set_style('darkgrid', {
        'axes.linewidth': 2,
        'legend.fancybox': True})

In [5]:
from PyHEADTAIL.trackers.rf_bucket import RFBucket
import PyHEADTAIL.particles.generators as gen

### Create bunches and bunches_list

In [6]:
momentum = 450e9 * e/c
gamma = np.sqrt((momentum/(m_p*c))**2 + 1)
beta = np.sqrt(1 - gamma**-2)

circumference = 26658.883
alpha = 3.225e-4
h1 = 35640
V1 = 8e6
p_increment = 0 * e/c * circumference/(beta*c)

print gamma

479.606062093


In [7]:
rfbucket = RFBucket(
    charge=e, mass=m_p, gamma=gamma,
    circumference=circumference,
    alpha_array=[alpha], p_increment=0,
    harmonic_list=[h1], voltage_list=[V1], phi_offset_list=[0])

In [8]:
dt = np.random.uniform(0, 10e-9, 3)
epsn_z = np.random.uniform(0, 0.5, 3)
print dt
print epsn_z

[  6.41871292e-09   7.19524847e-09   7.91539442e-09]
[ 0.41942041  0.11852963  0.49772888]


In [9]:
dt = (0, 2.5e-9, 7.5e-9)
epsn_z = (0.35, 0.25, 0.1)

In [11]:
bunches_list = [
    gen.ParticleGenerator(
        macroparticlenumber=1e5, intensity=1e11,
        charge=e, mass=m_p, gamma=gamma,
        circumference=26658.883,
        distribution_x=gen.gaussian2D(2e-6),
        distribution_y=gen.gaussian2D(2e-6),
        distribution_z=gen.RF_bucket_distribution(rfbucket, epsn_z=epsn_z[i])
    ).generate() for i in xrange(3)]

for i, b in enumerate(bunches_list):
    b.z -= dt[i] * (b.beta*c)
    b.dt = dt[i]

bunches = sum(bunches_list)
bunches.dt = dt[0]

*** Maximum RMS emittance 0.888689919173eV s.
... distance to target emittance: 2.32e-02
... distance to target emittance: 2.34e-02
... distance to target emittance: -5.11e-04
... distance to target emittance: 1.04e-05
--> Emittance: 0.350000004218
--> Bunch length:0.0697492609648
*** Maximum RMS emittance 0.888689919173eV s.
... distance to target emittance: 1.69e-02
... distance to target emittance: 1.70e-02
... distance to target emittance: -4.02e-05
--> Emittance: 0.250000069118
--> Bunch length:0.0580813886211
*** Maximum RMS emittance 0.888689919173eV s.
... distance to target emittance: 2.45e-03
... distance to target emittance: 2.57e-03
... distance to target emittance: 1.73e-06
--> Emittance: 0.100000001214
--> Bunch length:0.0359672667212


### Wakes and wake kicks and functions

In [12]:
from PyHEADTAIL.particles.slicing import UniformBinSlicer
from PyHEADTAIL.impedances.wakes import CircularResonator, WakeField

In [14]:
slicer = UniformBinSlicer(60, z_cuts=(-.3, .3))

wake = CircularResonator(R_shunt=1e9, frequency=1e9, Q=20)

wakefields = WakeField(slicer, wake)
wakekick = wakefields.wake_kicks[0]
wakefunction = wakekick.wake_function

*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!
*** PyHEADTAIL WARNING! Acceleration not handled properly by this kind of convolution due to changing bunch length!


### Convolutions

In [15]:
def convolution_python(t_target, t_source, c_source, w):
    dxp = 0.*t_target
    for k in xrange(len(t_target)):
        for l in xrange(len(t_source)):
            dxp[k] += c_source[l]*w(t_target[k]-t_source[l])

    return dxp

In [16]:
def convolution_numpy(t_target, t_source, c_source, w):
    tmin, tmax = t_source[0], t_source[-1]
    tt = np.concatenate((t_target-tmax, (t_target-tmin)[1:]))
    
    return np.convolve(c_source, w(tt), mode='valid')

In [17]:
def convolve_multibunch(times, moments, wf, dt=None, f_convolution=convolution_python):
    dxp = []
    if dt is None: dt = 0.*np.array(times)
    for i in xrange(len(times)):
        z = 0.*times[i]
        t_target = times[i]
        for j in range(i+1):
            t_source = times[j] + dt[i] - dt[j]
            c_source = moments[j]
            z += f_convolution(t_target, t_source, c_source, wf)
            
        dxp.append(z)
        
    return dxp

In [18]:
def convolve_PyHEADTAIL_sb(bunch, times, ages, cmoments, betas):
    return wakekick._accumulate_source_signal(bunch, times, ages, cmoments, betas)

In [19]:
def convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas):
    return wakekick._accumulate_source_signal_multibunch(bunches, times, ages, cmoments, betas)

### Use cases
#### Create slices and slices_list
---

In [21]:
def get_times_and_moments(bunches_list, slicer):
    bunches_list = np.atleast_1d(bunches_list)

    slices_list = []
    times_list = []
    cmoments_list = []
    for i, b in enumerate(bunches_list):
        distance = dt[i]*b.beta*c
        b.z += distance
        slices_list.append(b.get_slices(slicer))
        times_list.append(slices_list[-1].z_centers / (b.beta*c))
        cmoments_list.append(slices_list[-1].charge_per_slice)
        b.z -= distance

    # Multi-turn multi-bunch times and moments arrays - here, only one turn assumed
    n_turns = 1
    n_bunches = len(bunches_list)
    n_slices = len(times_list[0])

    tmp = np.zeros((n_turns, n_bunches, n_slices))
    tmp[0,:,:] = times_list
    times_array = tmp

    tmp = np.zeros((n_turns, n_bunches, n_slices))
    tmp[0,:,:] = cmoments_list
    cmoments_array = tmp

    ages_list = np.array([0 for b in bunches_list])
    betas_list = np.array([[b.beta for b in bunches_list]])
    
    return times_array, ages_list, cmoments_array, betas_list

#### Single beam

In [22]:
# Single beam
slicer = UniformBinSlicer(400, z_cuts=(-3, .5))

times, ages, cmoments, betas = get_times_and_moments(bunches, slicer)

delta_xp_mb = convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas)
# %timeit convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas)

times_list = times[:,0,:]
cmoments_list = cmoments[:,0,:]

delta_xp_sb = convolve_PyHEADTAIL_sb(bunches, times_list, ages, cmoments_list, betas)
delta_xp_np = convolve_multibunch(times_list, cmoments_list, wakefunction)
# %timeit convolve_PyHEADTAIL_sb(bunches, times_list, ages, cmoments_list, betas)
# %timeit convolve_multibunch(times_list, cmoments_list, wakefunction)
# %timeit convolve_multibunch(times_list, cmoments_list, wakefunction, f_convolution=convolution_numpy)

In [23]:
times = np.squeeze(times)
cmoments = np.squeeze(cmoments)
dxp1 = np.squeeze(delta_xp_mb)
dxp2 = np.squeeze(delta_xp_sb)
dxp3 = np.squeeze(delta_xp_np)

t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))

col = sns.color_palette('husl', 6)
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))

ax1.plot(t0, wakefunction(t0)[::-1]/max(wakefunction(t0)[::-1]))
ax1.plot(times-times[-1], cmoments/max(cmoments))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Bunch', 'Wake'])
ax1.set_xlim(-11e-9, 3e-9)

ax2.plot(dxp1/max(dxp1))
ax2.plot(dxp2/max(dxp2))
ax2.plot(-dxp3/max(-dxp3))
ax2.legend(['PyHEADTAIL MB', 'PyHEADTAIL SB', 'Notebook manual'])

ax3.plot(times-times[-1], 2*cmoments/max(cmoments))
ax3.plot(times-times[-1], dxp1/max(dxp1), label="all bunches")

ax3.legend()
ax3.set_xlim(-11e-9, 3e-9)
ax3.set_ylim(-2.1, 2.1)

(-2.1, 2.1)

#### Multiple bunches

In [24]:
slicer = UniformBinSlicer(60, z_cuts=(-.3, .3))

times_list, ages, cmoments_list, betas = get_times_and_moments(bunches_list, slicer)

delta_xp_mb_l = convolve_PyHEADTAIL_mb(bunches_list, times_list, ages, cmoments_list, betas)
# %timeit convolve_PyHEADTAIL_mb(bunches, times, ages, cmoments, betas)

delta_xp_sb_l = []
delta_xp_np_l = []
for i, b in enumerate(bunches_list):
    delta_xp_sb_l.append(convolve_PyHEADTAIL_sb(
            b, times_list[:,i,:], ages, cmoments_list[:,i,:], betas))
    delta_xp_np_l.append(convolve_multibunch(
            times_list[:,i,:], cmoments_list[:,i,:], wakefunction, f_convolution=convolution_numpy))

In [26]:
times = np.squeeze(times)
cmoments = np.squeeze(cmoments)
dxp1 = np.squeeze(delta_xp_mb)
dxp2 = np.squeeze(delta_xp_sb)
dxp3 = np.squeeze(delta_xp_np)

dxp1_l = np.squeeze(delta_xp_mb_l)
dxp2_l = np.squeeze(delta_xp_sb_l)
dxp3_l = np.squeeze(delta_xp_np_l)

t0 = np.concatenate((times-times[-1], (times-times[0])[1:]))

col = sns.color_palette('husl', 7)
fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(12,10))

ax1.plot(t0, wakefunction(t0)[::-1]/max(wakefunction(t0)[::-1]))
ax1.plot(times-times[-1], cmoments/max(cmoments))
ax1.set_ylim((-1.1, 1.1))
ax1.legend(['Bunch', 'Wake'])
ax1.set_xlim(-11e-9, 3e-9)

ax2.plot(dxp1/max(dxp1))
ax2.plot(dxp2/max(dxp2))
ax2.plot(-dxp3/max(-dxp3))
ax2.legend(['PyHEADTAIL MB', 'PyHEADTAIL SB', 'Notebook manual'])

ax3.plot(times-times[-1], 2*cmoments/max(cmoments), '--')
ax3.plot(times-times[-1], dxp1/max(dxp1), '--', label="all bunches")

labels = ["first bunch", "second bunch", "third bunch"]
for i in xrange(len(bunches_list)):
    tml = np.squeeze(times_list[:,i,:])
    cml = np.squeeze(cmoments_list[:,i,:])
    ax3.plot(tml-times[-1]-dt[i], 2*cml/np.max(cmoments_list), c=col[i], label=labels[i])
    ax3.plot(tml-times[-1]-dt[i], dxp1_l[i]/max(dxp1), c=col[i+3])

#     [ax3.axvline(s, c=c[0], alpha=0.8) for s in (dts-dts[-1])]
#     [ax3.axvline(s, c=c[1], alpha=0.8) for s in (dts-dts[-1]-dt[i])]
#     [ax3.axvline(s, c=c[2], alpha=0.8) for s in (alltimes-alltimes[-1])]

ax3.legend()
ax3.set_xlim(-11e-9, 3e-9)
ax3.set_ylim(-2.1, 2.1)

(-2.1, 2.1)

In [21]:
plt.close('all')